# Week 8 Bank Data Case Study

## Load Packages

In [1]:
import pandas as pd
import numpy as np

## Read the Data

In this section we read in the data. 

In [2]:
df = pd.read_csv("../../../case_8.csv")
df.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,1.630686,7.464411,C,4.145098,9.191265,2.436402,2.483921,2.301630,...,6.822439,3.549938,0.598896,AF,1.672658,3.239542,1.957825,0,1.925763,1.739389
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,1.630686,7.464411,C,4.145098,8.742359,2.436402,2.483921,1.496569,...,6.822439,3.549938,0.919812,Z,1.672658,3.239542,2.030373,0,1.925763,1.739389


No obvious issues like parsing errors or missings. Lets see what we have for data types.

In [3]:
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114321 entries, 0 to 114320
Data columns (total 133 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ID      int64  
 1   target  int64  
 2   v1      float64
 3   v2      float64
 4   v3      object 
 5   v4      float64
 6   v5      float64
 7   v6      float64
 8   v7      float64
 9   v8      float64
 10  v9      float64
 11  v10     float64
 12  v11     float64
 13  v12     float64
 14  v13     float64
 15  v14     float64
 16  v15     float64
 17  v16     float64
 18  v17     float64
 19  v18     float64
 20  v19     float64
 21  v20     float64
 22  v21     float64
 23  v22     object 
 24  v23     float64
 25  v24     object 
 26  v25     float64
 27  v26     float64
 28  v27     float64
 29  v28     float64
 30  v29     float64
 31  v30     object 
 32  v31     object 
 33  v32     float64
 34  v33     float64
 35  v34     float64
 36  v35     float64
 37  v36     float64
 38  v37     float64
 39  v38     int64  
 40  v

We see mostly floats. There are some object data types we should probably recast. We have 114K observations, plenty to work with.  No variable names, as expected. He said there are no missings but lets check anyway.

In [4]:
df.isnull().values.any()

False

Fine. He told the truth. How about that target variable

In [5]:
counts = df.target.value_counts()
print(counts)
print(round(counts[0]/sum(counts),4))

1    87021
0    27300
Name: target, dtype: int64
0.2388


The target is binary and a little unbalanced, but not terrible.

## Data Cleaning

boring

In [ ]:
#data['minimum_nights'] = data['minimum_nights'].astype('float')

## EDA

boring

## Modeling Prep

In [14]:
def transform_data(data):
    #OH encode
    label_encode = [
        x for x, y in dict(data.dtypes).items()
        if type(y) == pd.CategoricalDtype
    ]
    for var in label_encode:
        #impute to Mode
        print(f"{var} was OHE'd")
        data[var].fillna(data[var].mode()[0], inplace=True)
        cat_list = pd.get_dummies(data[var], prefix=var)
        data_new = data.join(cat_list)
        data = data_new

    normalize = [x for x, y in dict(data.dtypes).items() if y == np.float64]
    for col in normalize:
        #change impution scheme?
        print(f"{col} was Standard Scaled")
        data[col].fillna(data[col].median(), inplace=True)
        normalizer = StandardScaler()
        ar_train = data[col].to_numpy().reshape(-1, 1)
        normalizer.fit(ar_train)
        data.loc[:, col] = normalizer.transform(ar_train)
    
    data.drop(columns=label_encode, inplace=True)
    #data.drop(columns=target_col, inplace=True)
    
    return data

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score #https://scikit-learn.org/stable/modules/model_evaluation.html

In [7]:
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

In [8]:
n_estimators= list(range(80, 110, 10))
max_features = list(range(5, 50, 5))
min_samples_split = list(range(500, 701, 100))
min_samples_leaf = [10, 20]
print(f'n_estimator_grid_search:{n_estimators}')
print(f'max_features_grid_search:{max_features}')
print(f'min_samples_split_grid_search:{min_samples_split}')
print(f'min_samples_leaf_grid_search:{min_samples_leaf}')


param_dist = {'n_estimators': n_estimators,
              'max_features': max_features,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf}

scoring = {
            'Accuracy':'accuracy'
            , 'F-1 Score':'f1'
            , 'Log Loss':'neg_log_loss'}

n_estimator_grid_search:[80, 90, 100]
max_features_grid_search:[5, 10, 15, 20, 25, 30, 35, 40, 45]
min_samples_split_grid_search:[500, 600, 700]
min_samples_leaf_grid_search:[10, 20]


In [9]:
X = df.copy().drop(columns=["ID","target"]).select_dtypes(include=['number'])
print("The shape of X is: ", X.shape)

y = df.loc[:,"target"].copy()
print("The shape of y is: ", y.shape)

The shape of X is:  (114321, 112)
The shape of y is:  (114321,)


## Random Forest
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pickle
clf = RandomForestClassifier(max_depth=2, random_state=0, n_jobs=-1)
clf.fit(X, y)

D:\Users\William\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

Grid search for random forest

In [11]:
%%time
#%%script false --no-raise-error

n_iter_search = 2
rf_random_search = RandomizedSearchCV(clf, param_distributions=param_dist, scoring=scoring, cv = cv, random_state=42,
                                   n_iter=n_iter_search, refit='Accuracy')
rf_random_search.fit(X, y)

filename = 'rf_random_search.p'
pickle.dump(rf_random_search, open(filename, 'wb'))

Wall time: 56 s


In [12]:
rf_random_search = pd.read_pickle('rf_random_search.p')
pd.DataFrame(rf_random_search.cv_results_)


D:\Users\William\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_Accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Users\William\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_Accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Users\William\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_Accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Users\William\Anaconda3\lib\site-pack

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_features,params,split0_test_Accuracy,...,mean_test_Log Loss,std_test_Log Loss,rank_test_Log Loss,split0_train_Log Loss,split1_train_Log Loss,split2_train_Log Loss,split3_train_Log Loss,split4_train_Log Loss,mean_train_Log Loss,std_train_Log Loss
0,4.877302,0.840360,0.376586,0.000893,100,600,20,45,"{'n_estimators': 100, 'min_samples_split': 600...",0.761207,...,-0.511646,0.000566,1,-0.510321,-0.510189,-0.510482,-0.510396,-0.510773,-0.510432,0.000196
1,3.229324,0.051449,0.377685,0.004225,90,500,10,35,"{'n_estimators': 90, 'min_samples_split': 500,...",0.761207,...,-0.514810,0.000694,2,-0.513723,-0.513504,-0.513954,-0.513783,-0.513657,-0.513724,0.000148


## XGBoost

In [13]:
from xgboost import XGBClassifier
#https://xgboost.readthedocs.io/en/latest/build.html

D:\Users\William\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [14]:
# A parameter grid for XGBoost
params = {
        'learning_rate': [0.005, 0.01, 0.02, 0.05, 0.1],
        'n_estimators': [100,200,400,600,800,1000],
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

xgb = XGBClassifier(objective='binary:logistic',
                    silent=True, nthread=-1)

In [15]:
%%time
#%%script false --no-raise-error
n_iter_search = 2
xgb_random_search = RandomizedSearchCV(xgb, param_distributions=params, scoring=scoring, cv = cv, random_state=42,
                                   n_iter=n_iter_search, refit='Accuracy')
xgb_random_search.fit(X, y)

filename = 'xgb_random_search.p'
pickle.dump(xgb_random_search, open(filename, 'wb'))

Wall time: 11min 39s


In [16]:
xgb_random_search = pd.read_pickle('xgb_random_search.p')
pd.DataFrame(xgb_random_search.cv_results_)

D:\Users\William\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_Accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Users\William\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_Accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Users\William\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_Accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
D:\Users\William\Anaconda3\lib\site-pack

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_min_child_weight,param_max_depth,param_learning_rate,param_gamma,...,mean_test_Log Loss,std_test_Log Loss,rank_test_Log Loss,split0_train_Log Loss,split1_train_Log Loss,split2_train_Log Loss,split3_train_Log Loss,split4_train_Log Loss,mean_train_Log Loss,std_train_Log Loss
0,78.695948,0.513996,0.563534,0.012532,0.8,1000,5,5,0.1,2,...,-0.499054,0.001442,2,-0.371650,-0.371244,-0.370991,-0.371692,-0.370583,-0.371232,0.000416
1,37.084821,0.255249,0.416493,0.004925,1,800,10,3,0.005,1,...,-0.496882,0.000619,1,-0.493681,-0.493714,-0.493701,-0.493410,-0.494110,-0.493723,0.000224


## SVC
This never finishes. Need to reduce features for it to do anything.

In [69]:
#from sklearn.svm import SVC
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [68]:
#param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf']}

In [ ]:
%%time
#%%script false --no-raise-error
#n_iter_search = 2
#svc_random_search = RandomizedSearchCV(SVC(), param_distributions=param_grid, scoring=scoring, cv = 2, random_state=42,
                                   n_iter=n_iter_search, refit='Accuracy')
#svc_random_search.fit(X, y)

#filename = 'svc_random_search.p'
#pickle.dump(svc_random_search, open(filename, 'wb'))

## Linear SVC

In [17]:
from sklearn.svm import LinearSVC
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

In [25]:
param_grid = {'C':[1,10,100,1000], 'loss':['hinge','square_hinge']}

In [26]:
n_iter_search = 2
lsvc_random_search = RandomizedSearchCV(LinearSVC(), param_distributions=param_grid, scoring=scoring, cv = 2, random_state=42,
                                   n_iter=n_iter_search, refit='Accuracy')
lsvc_random_search.fit(X, y)

filename = 'lsvc_random_search.p'
pickle.dump(lsvc_random_search, open(filename, 'wb'))

D:\Users\William\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Unsupported set of arguments: loss='square_hinge' is not supported, Parameters: penalty='l2', loss='square_hinge', dual=True